## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-24-15-53-49 +0000,nypost,Vanity Fair to publish ‘abstract nude portrait...,https://nypost.com/2025/11/24/media/vanity-fai...
1,2025-11-24-15-53-27 +0000,nypost,Russian ‘Popeye’ who injected his biceps to gr...,https://nypost.com/2025/11/24/world-news/freak...
2,2025-11-24-15-49-06 +0000,nypost,Mother charged with manslaughter after diabeti...,https://nypost.com/2025/11/24/health/mother-ch...
3,2025-11-24-15-44-33 +0000,bbc,US presses Europe on taxes on big tech companies,https://www.bbc.com/news/articles/clydjnlm54po...
4,2025-11-24-15-43-30 +0000,nypost,"Flooding, severe storms soak Texas as 73M peop...",https://nypost.com/2025/11/24/us-news/flooding...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2384/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
97,ukraine,21
134,trump,18
104,peace,15
164,plan,14
53,new,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
127,2025-11-24-07-31-06 +0000,nyt,"Ukraine, U.S. Meet About Trump’s Peace Plan to...",https://www.nytimes.com/2025/11/23/world/europ...,95
205,2025-11-23-17-38-05 +0000,nypost,Zelensky thanks Trump for support after critic...,https://nypost.com/2025/11/23/world-news/zelen...,80
31,2025-11-24-14-24-48 +0000,nyt,U.S. and Ukraine Hail Progress on Peace Plan t...,https://www.nytimes.com/2025/11/24/world/europ...,79
126,2025-11-24-07-32-21 +0000,nyt,What Is Trump’s 28-Point Plan to End Russia’s ...,https://www.nytimes.com/2025/11/22/world/europ...,79
164,2025-11-23-23-12-21 +0000,nypost,Leaked EU peace deal calls for Ukraine to keep...,https://nypost.com/2025/11/23/world-news/leake...,75


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
127,95,2025-11-24-07-31-06 +0000,nyt,"Ukraine, U.S. Meet About Trump’s Peace Plan to...",https://www.nytimes.com/2025/11/23/world/europ...
217,35,2025-11-23-16-49-58 +0000,cbc,"Carney, Modi agree to launch negotiations on n...",https://www.cbc.ca/news/politics/carney-modi-n...
62,32,2025-11-24-11-58-48 +0000,nypost,Outgoing Rep. Marjorie Taylor Greene shoots do...,https://nypost.com/2025/11/24/us-news/outgoing...
194,32,2025-11-23-19-31-39 +0000,bbc,Israel kills top Hezbollah official in first a...,https://www.bbc.com/news/articles/cn81j54xjx1o...
67,29,2025-11-24-11-39-26 +0000,nypost,Erika Kirk reveals she prayed she was pregnant...,https://nypost.com/2025/11/24/us-news/erika-ki...
86,29,2025-11-24-11-00-00 +0000,wapo,More Americans are getting their power shut of...,https://www.washingtonpost.com/business/2025/1...
6,26,2025-11-24-15-40-02 +0000,nypost,‘Adult accomplice’ busted with ‘Slender Man’ s...,https://nypost.com/2025/11/24/us-news/woman-42...
175,25,2025-11-23-21-41-00 +0000,nypost,Glam former prime minister Sanna Marin complai...,https://nypost.com/2025/11/23/world-news/glam-...
78,24,2025-11-24-11-00-00 +0000,latimes,"'Legend': Claire Rothman, Forum president duri...",https://www.latimes.com/california/story/2025-...
0,22,2025-11-24-15-53-49 +0000,nypost,Vanity Fair to publish ‘abstract nude portrait...,https://nypost.com/2025/11/24/media/vanity-fai...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
